In [1]:
from lusidtools.jupyter_tools import toggle_code

# Computing P&L and Handling Dividends for Equities

In this notebook, we demonstrate how P&L can be calculated for Equity instruments as well as how to book in dividends to our cash position. 
In this example, we will be using Microsoft shares.

## Table of Contents:
* [1. Create a portfolio](#1.-Create-Portfolio)
* [2. Creating an equity instrument](#2.-Create-an-Equity-Instrument)
* [3. Transactions](#3.-Transactions)
* [4. Quotes](#4.-Quotes)
* [5. Creating a corporate action](#5.-Create-Dividend-Corporate-Action)
* [6. Valuations](#6.-Valuations)

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
from utilities.formatting_tools import cashladder_to_df

# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.8212.0


In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)
corporate_action_sources_api = api_factory.build(lu.api.CorporateActionSourcesApi)
property_definitions_api = api_factory.build(lu.api.PropertyDefinitionsApi)

In [4]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
ca_code = "ibor_corp_act"
portfolio_code = "EquityPortfolioForPnLCalc"

# 1. Create Portfolio

We must first create a corporate action source to link to the portfolio we plan to keep our equity in. This will allow us to perform a valuation and inspect the cash flows at a later stage.

In [5]:
try:

    source_request = lm.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_code,
        display_name="Ibor Corporate Action Source",
        description="Corporate Actions source for sample notebook",
    )

    source_result = api_factory.build(
        lu.api.CorporateActionSourcesApi
    ).create_corporate_action_source(
        create_corporate_action_source_request=source_request
    )

except:
    print("Already Exists")

Already Exists


In [6]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
            corporate_action_source_id=lu.ResourceId(scope=scope, code=ca_code),
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id EquityPortfolioForPnLCalc because it already exists in scope ibor.


# 2. Create an Equity Instrument

We must first create a property definition for the dividend yield property we are planning on using later in this notebook.

In [7]:
try:
    property_definition_request = lm.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="dividend_yield",
        display_name="Dividend Yield",
        data_type_id=lm.ResourceId(
            scope="system",
            code="number",
        ),
        life_time="Perpetual",
    )

    property_definitions_api.create_property_definition(
        create_property_definition_request=property_definition_request
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/ibor/dividend_yield' because it already exists.


We create a function that takes in our equity variables and upserts an equity to LUSID.

In [8]:
def create_equity(
    name,
    client_internal,
    dom_ccy,
    dividend_yield,
):

    equity = lm.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy=dom_ccy,
        asset_class="Equities",
        simple_instrument_type="Equity",
    )

    # properties = lm.InstrumentProperties()
    properties = lm.ModelProperty(
        key=f"Instrument/{scope}/dividend_yield",
        value=lm.PropertyValue(
            metric_value=lm.MetricValue(
                value=dividend_yield,
                # unit="Decimal",
            )
        ),
    )

    equity_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(client_internal)},
        definition=equity,
        properties=[properties],
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    equity_luid = upsert_response.values[client_internal].lusid_instrument_id
    print(equity_luid)

# 2.1 Upsert an Equity

We can now set the variables of the equity and upsert it into LUSID using the function we built in the cell above.

In [9]:
name = "Microsoft"
identifier = "MSFT"
dom_ccy = "USD"
dividend_yield = 0.88

create_equity(name, identifier, dom_ccy, dividend_yield)

LUID_FP69LVAZ


# 3. Transactions

## 3.1 Create Transaction Request

Once the equity has been created, we can generate a transaction that adds it to our portfolio.

In [10]:
transactions = pd.read_csv("data/equity_transaction_data.csv")
transactions

,txn_id,type,Isin,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,StockIn,US5949181045,MSFT,2021-09-01T10:00:00Z,2021-09-02T10:00:00Z,1000,300,300000,USD,EquityPortfolioForPnLCalc


In [11]:
for portfolio_code, grouped_df in transactions.groupby("portfolio"):

    transaction_request = [
        lm.TransactionRequest(
            transaction_id=row["txn_id"],
            type=row["type"],
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["client_id"],
                "Instrument/default/Isin": row["Isin"],
            },
            transaction_date=row["trade_date"],
            settlement_date=row["settlement_date"],
            units=row["quantity"],
            transaction_price=lm.TransactionPrice(price=row["price"], type="Price"),
            total_consideration=lm.CurrencyAndAmount(
                amount=row["total_consideration"], currency=row["currency"]
            ),
        )
        for index, row in grouped_df.iterrows()
    ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=transaction_request
    )

    print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2021-11-10 13:08:26.100668+00:00


# 4. Quotes

The equity is in our portfolio, but currently lacks pricing. We will thus upsert quotes for the equity that we retrieved from our market data system. We upload prices from when the equity was bought until today (at the time of writing, 9 November 2021). Below we can see the last 5 of the ~50 quotes that were uploaded.

In [12]:
equity_prices = pd.read_csv("data/equity_quotes_data.csv")
equity_prices.tail()

,Luid,date,price,currency
47,LUID_FP69LVAZ,2021-11-08,336.99,USD
48,LUID_FP69LVAZ,2021-11-09,337.05,USD
49,LUID_FP69LVAZ,2021-11-10,338.05,USD
50,LUID_FP69LVAZ,2021-11-11,339.05,USD
51,LUID_FP69LVAZ,2021-11-12,340.05,USD


In [13]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["Luid"],
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
    )
    for index, row in equity_prices.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quote successfully loaded into LUSID. 52 quotes loaded.


# 5. Create Dividend Corporate Action

To create our corporate action, we define an input and an output transition. Our input transition is the number of shares in Microsoft, denoted by the Lusid Instrument ID "LUID_FP69LVAZ". The output transition is the amount of USD per share we will receive. The result is that for every 1 share, we receive 0.56 USD. You can structure all sorts of corporate actions in this way. For our example, we structured a divident payment. 

In [14]:
transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/LusidInstrumentId": "LUID_FP69LVAZ"
            },
            units_factor=1,
            cost_factor=1,
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={"Instrument/default/Currency": "USD"},
                units_factor=0.56,
                cost_factor=1,
            )
        ],
    )
]

dividend = lm.UpsertCorporateActionRequest(
    corporate_action_code=ca_code,
    announcement_date=datetime(2021, 9, 2, 00, tzinfo=pytz.utc),
    ex_date=datetime(2021, 9, 3, 00, tzinfo=pytz.utc),
    record_date=datetime(2021, 9, 4, 00, tzinfo=pytz.utc),
    payment_date=datetime(2021, 9, 9, 00, tzinfo=pytz.utc),
    transitions=transitions,
)

corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[dividend]
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-ci.lusid.com/api/api/schemas/entities/UpsertCorporateActionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMDF9RN77740:0000000C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'ibor_corp_act': {'announcement_date': datetime.datetime(2021, 9, 2, 0, 0, tzinfo=tzutc()),
                              'corporate_action_code': 'ibor_corp_act',
                              'description': None,
                              'ex_date': datetime.datetime(2021, 9, 3, 0, 0, tzinfo=tzutc()),
                              'payment_date': datetime.datetime(2021, 9, 9, 0, 0, tzinfo=tzutc()),
                              'record_date': da

# 6. Valuations

Once we have the equity booked into a portfolio, we can now value this portfolio. The recipe below describes how we will go about valuing the instruments in our portfolio (in this case just our equity).

## 6.1 Create valuation recipe

In [15]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="equityValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 6.2 Create daily valuation function

Once we have made a recipe, we can now create a function that outputs a dataframe with the valuation of our portfolio. We choose to display PV, PnL and some instrument identifiers.

In [16]:
def get_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="equityValuation"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
            lm.AggregateSpec("Instrument/ibor/dividend_yield", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Instrument/ibor/dividend_yield": "Dividend Yield",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

We will now display the first few days in the lifecycle of our equity. Recall that we have bought this equity on the 1st of September 2021.

## Day 1

At 10AM on day 1, we value our equity position and find that it is worth 300,000 USD.

In [17]:
get_val("2021-09-01T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,PnL (1-day),Dividend Yield
0,"300,000.00",Microsoft,MSFT,300.00,"1,000.00",None,0.88


## Day 2

On day 2 the share price of MSFT moves to 301.15, giving us a 1 day PnL of 1,150 USD.

In [18]:
get_val("2021-09-02T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,PnL (1-day),Dividend Yield
0,"301,150.00",Microsoft,MSFT,301.15,"1,000.00","1,150.00",0.88


## Cash Ladder

If we take a look at the cash ladder between our first day of owning this equity, 1 September 2021, and today (11 November 2021 at time of writing) we can see all cash flows associated with this position so far.

In [19]:
cash_ladder = transaction_portfolios_api.get_portfolio_cash_ladder(
    scope=scope,
    code=portfolio_code,
    effective_at="2021-11-11T10:00:00Z",
    from_effective_at="2021-09-01T10:00:00Z",
    to_effective_at="2021-11-11T10:00:00Z",
)

cashladder_to_df(cash_ladder)

,Currency,Date,Activity,Value
0,USD,2021-09-09 00:00:00+00:00,Open,0.00
1,USD,2021-09-09 00:00:00+00:00,CorporateActionTransition,560.00
2,USD,2021-09-09 00:00:00+00:00,Close,560.00


## Dividend Date
In the cash ladder, we can see that our dividend comes in on the 9th of September 2021.

We run our valuation again for the dividend date and find that we have received a dividend payment of 0.56 USD per share. This results in an cash position of 560 USD.

In [20]:
get_val("2021-09-09T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,PnL (1-day),Dividend Yield
0,"297,250.00",Microsoft,MSFT,297.25,"1,000.00","-2,960.00",0.88
1,560.00,USD,None,nan,560.00,0.00,nan
